In [ ]:
from flask import Flask, render_template, Response
import cv2
import face_recognition
import os
from datetime import datetime
from werkzeug.serving import run_simple

app = Flask(__name__)

# Load known faces and their names
known_face_encodings = []
known_face_names = []

# Path to the directory containing known faces
known_faces_dir = 'C:/Users/admin/Documents/anaconda/surveillance_project/known_faces'

# Load each face in the directory
for filename in os.listdir(known_faces_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(known_faces_dir, filename)
        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])

# Directories to save detected faces
save_dir_known = 'C:/Users/admin/Documents/anaconda/surveillance_project/workers'
save_dir_unknown = 'C:/Users/admin/Documents/anaconda/surveillance_project/customers'
if not os.path.exists(save_dir_known):
    os.makedirs(save_dir_known)
if not os.path.exists(save_dir_unknown):
    os.makedirs(save_dir_unknown)

saved_faces = set()
customer_encodings = []
customer_count = 1

def generate_frames():
    webcam = cv2.VideoCapture(0)
    while True:
        success, video = webcam.read()
        if not success:
            break
        else:
            small_frame = cv2.resize(video, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                else:
                    customer_match = face_recognition.compare_faces(customer_encodings, face_encoding)
                    if True in customer_match:
                        match_index = customer_match.index(True)
                        name = f"customer{match_index + 1}"
                    else:
                        name = f"customer{customer_count}"
                        customer_encodings.append(face_encoding)
                        customer_count += 1

                cv2.rectangle(video, (left, top), (right, bottom), (0, 255, 0), 3)
                cv2.rectangle(video, (left, top - 35), (right, top), (0, 255, 0), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(video, name, (left + 6, top - 6), font, 1.0, (255, 255, 255), 1)

                if name not in saved_faces:
                    face_roi = video[top:bottom, left:right]
                    date_dir = os.path.join(save_dir_known if name in known_face_names else save_dir_unknown, datetime.now().strftime('%Y-%m-%d'))
                    if not os.path.exists(date_dir):
                        os.makedirs(date_dir)
                    cv2.imwrite(os.path.join(date_dir, name + '_' + datetime.now().strftime("%H-%M-%S") + '.jpg'), face_roi)
                    saved_faces.add(name)

            ret, buffer = cv2.imencode('.jpg', video)
            video_frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + video_frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    run_simple('localhost', 5000, app)


In [ ]:
import cv2
import time
trained = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
webcam = cv2.VideoCapture(0)
start_time = time.time()
while True:
    working, video = webcam.read()
    b_w = cv2.cvtColor(video, cv2.COLOR_BGR2GRAY)
    face = trained.detectMultiScale(b_w)
    for(x, y, w, h) in face:
        cv2.rectangle(video,(x, y), (x+w, y+h),(0, 255, 0), 3)
    cv2.imshow("Live_video",video)
    key = cv2.waitKey(1)
    if time.time() - start_time > 5:
        break
    if key == ord('q'):
        break

# Release the webcam and close any open windows
webcam.release()
cv2.destroyAllWindows()